In [1]:
ls cache

cache_chunk.npy  pred_inst.npy  pred_map.npy


In [2]:
from dataloader.infer_loader import SerializeArray
from torch.utils import data
from tqdm import tqdm
from infer.wsi import _get_chunk_patch_info
import numpy as np

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
wsi_proc_shape = [81590, 83590] 
chunk_input_shape = [20000, 20000] 
patch_input_shape = np.array([256, 256])
patch_output_shape = np.array([164, 164])

In [46]:
wsi_proc_shape[0]/4, wsi_proc_shape[1]/4

(20397.5, 20897.5)

In [47]:
chunk_info_list, patch_info_list = _get_chunk_patch_info(
            wsi_proc_shape,
            chunk_input_shape,
            patch_input_shape,
            patch_output_shape,
        )

In [48]:
chunk_info_list.shape, patch_info_list.shape

((25, 2, 2, 2), (253470, 2, 2, 2))

In [49]:
from misc.wsi_handler import get_file_handler
import pathlib

wsi_path = 'dataset/sample_wsis/img/B_37_FB3-SL_570-ST_NISL-SE_1708_lossless.dat'

path_obj = pathlib.Path(wsi_path)
wsi_ext = path_obj.suffix
wsi_name = path_obj.stem

wsi_handler = get_file_handler(wsi_path, backend=wsi_ext)
 

In [50]:
import time

In [74]:
cache_path = 'cache'

masking = lambda x, a, b: (a <= x) & (x <= b)

for idx in range(0, chunk_info_list.shape[0]):
    chunk_info = chunk_info_list[idx]
    # select patch basing on top left coordinate of input
    start_coord = chunk_info[0, 0]
    end_coord = chunk_info[0, 1] - patch_input_shape
    selection = masking(
        patch_info_list[:, 0, 0, 0], start_coord[0], end_coord[0]
    ) & masking(patch_info_list[:, 0, 0, 1], start_coord[1], end_coord[1])
    chunk_patch_info_list = np.array(
        patch_info_list[selection]
    )  # * do we need copy ?
    
    chunk_patch_info_list -= chunk_info[:, 0]
    
    read_start = time.perf_counter()
    chunk_data = wsi_handler.read_region(
        chunk_info[0][0][::-1], (chunk_info[0][1] - chunk_info[0][0])[::-1]
    )
    chunk_data = np.array(chunk_data)[..., :3]
    np.save("%s/cache_chunk.npy" % cache_path, chunk_data)
    
    print('read time:',time.perf_counter()-read_start)
    
    patch_top_left_list = chunk_patch_info_list[:,0,0]
    
    print(patch_top_left_list.shape)
    
    dataset = SerializeArray(
        "%s/cache_chunk.npy" % cache_path,
        patch_top_left_list,
        patch_input_shape,
    )

#     print(len(dataset))
    
#     elt = dataset[0]
    
#     print(type(elt[0]),elt[0].shape,elt[1])

    if False:
        iter_start = time.perf_counter()
        for ii in range(len(dataset)):
            itm = dataset[ii]
        print('loop time', time.perf_counter()-iter_start)
    
    if not False:
        dataloader = data.DataLoader(
            dataset,
            num_workers=1,
            batch_size=132,
            drop_last=True,
        #             pin_memory = True
        )
        iter_start = time.perf_counter()
#         print(len(dataloader))
        for _,elt in enumerate(dataloader):
            pass
        print('iter time', time.perf_counter()-iter_start)

read time: 0.9534679441712797
(14641, 2)
iter time 5.994085043668747
read time: 1.0680331396870315
(14641, 2)
iter time 5.849077029153705
read time: 1.0795318414457142
(14641, 2)
iter time 5.099837225861847
read time: 1.0822388278320432
(14641, 2)
iter time 5.69696590770036
read time: 0.2525937049649656
(1452, 2)
iter time 1.863763720728457
read time: 0.9474504110403359
(14641, 2)
iter time 5.559365447610617
read time: 1.0643518920987844
(14641, 2)
iter time 6.315913878846914
read time: 1.0767146227881312
(14641, 2)
iter time 5.032477863132954
read time: 1.0773235382512212
(14641, 2)
iter time 6.120251965709031
read time: 0.25312652811408043
(1452, 2)
iter time 1.954146672040224
read time: 0.9629951808601618
(14641, 2)
iter time 5.89236883725971
read time: 1.0907739591784775
(14641, 2)
iter time 6.153984329197556
read time: 1.083296790253371
(14641, 2)
iter time 5.367110386956483
read time: 1.119552276097238
(14641, 2)
iter time 6.028979624155909
read time: 0.24550405330955982
(1452, 2

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 143, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 143, in <listcomp>
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibil

In [75]:
len(dataset),len(dataloader)

(3025, 22)

In [76]:
len(dataset)/132

22.916666666666668

In [77]:
for ii,elt in enumerate(dataloader):
    pass

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 143, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 143, in <listcomp>
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibil

In [78]:
ii

10

In [79]:
v = next(iter(dataloader))

terminate called without an active exception


2